## Experiment 2: Ion Barrier Switching Mode
This experiment determines how the particle beam behaves as the ion barrier goes from active to inactive.

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import libem
import numpy as np
from libvis import Visualizations, VideoMaker
import matplotlib.pyplot as plt

from components import *

from scipy.optimize import root_scalar

**General Configuration**

In [ ]:
def generate_barrier_space(V_out=1, V_in=-1, grate_dist=0.5, grate_sp=0.2, th=0.05, scale=30):
    sim = libem.EMSimulationSpace3D(space_size=(4, 2, 2), top_left=(-2, -1, -1), scale=scale)
    sim.compute(make_enforcer(
        enf(EMObjects.planar_mesh_3d, (grate_dist, -0.8, -0.8), 0, (0.1, 1.6, 1.6), (grate_sp, grate_sp), th, V_out),
        enf(EMObjects.planar_mesh_3d, (0, -0.8, -0.8), 0, (0.1, 1.6, 1.6), (grate_sp, grate_sp), th, V_in),
        enf(EMObjects.planar_mesh_3d, (-grate_dist, -0.8, -0.8), 0, (0.1, 1.6, 1.6), (grate_sp, grate_sp), th, V_out),
    ))
    sim.get_efield()
    return sim

NPARTICLES = 20
THICKNESS = 0.05
DISTANCE = 0.4
SPACING = 0.5
SCALE = 20

particle_opts = {
    "mass": (1, 0.25),
    "charge": (1, 0.25),
    "initial_loc": ((1.9, 0, 0), (0, 0, 0)),
    "initial_v": ((-1, 0, 0), (0.2, 0.1, 0.1)),
    "bounce_coef": 0
}

**Simulate Particle Beam**

In [ ]:
particles = libem.ChargedParticle3D.generate_particles(NPARTICLES, None,
                                                       particle_opts["mass"][0],
                                                       particle_opts["charge"][0],
                                                       particle_opts["initial_loc"][0],
                                                       particle_opts["initial_v"][0],
                                                       particle_opts["mass"][1],
                                                       particle_opts["charge"][1],
                                                       particle_opts["initial_loc"][1],
                                                       particle_opts["initial_v"][1],
                                                       particle_opts["bounce_coef"])
particles = [p for p in particles if p.charge > 0]

max_qke = max([abs(p.charge) for p in particles]) * 0.5 * max([p.mass for p in particles]) * \
            (max([np.linalg.norm(p.initial_velocity) for p in particles]))**2

voltage_range = np.arange(0, 1.1 * max_qke, max_qke / 30)[::-1]
passthrough = np.zeros(voltage_range.shape)
coherence = np.zeros(voltage_range.shape)

fig, axes = plt.subplots(3, 1, figsize=(12, 12))
plt.show()
video = VideoMaker(fig, axes)

print("Computing")

for i, voltage in enumerate(voltage_range):
    sim = generate_barrier_space(0, voltage, DISTANCE, SPACING, THICKNESS, SCALE)
    
    passed = []
    passed_v = []
    for j, p in enumerate(particles):
        p.sim = sim
        p.compute_motion((0, (p.initial_location[0] + 1.5) / abs(p.initial_velocity[0])))
        if p.position[0][-1] < -DISTANCE:
            passed.append(p.position[:,-1])
            passed_v.append(p.velocity[:,-1])
        print("\rVoltage " + str(round(voltage, 2)) + ", Particle " + str(j+1).rjust(3), end="")
            
    if len(passed) > 0:
        passed = np.array(passed)
        passed_v = np.array(passed_v)
        passthrough[i] = passed.shape[0] / NPARTICLES
        extrapolate_times = (passed[:,0] + 30.0) / np.abs(passed_v[:,0])
        extrapolated_locs = np.array(
            [(extrapolate_times[i] * passed_v[i,1:]) +  passed[i,1:] for i in range(passed.shape[0])])
        coherence[i] = np.average(np.linalg.norm(extrapolated_locs, axis=1))
    
    video.new_frame()
        
    axes[0].set_xlabel("X")
    axes[0].set_ylabel("Z")
    if voltage != 0:
        sim2d = libem.EMSimulationSpace2D.from_3d(sim, axis=1)
        Visualizations.colormesh_2d(sim2d, color_norm=voltage, graph_ax=axes[0])
    for p in particles:
        Visualizations.trajectory_2d(p.time, p.position, axis=1, graph_ax=axes[0])
    axes[0].set_xlim((-2, 2))
    axes[0].set_ylim((-1, 1))
        
    axes[1].set_xlabel("Voltage")
    axes[1].set_ylabel("Passthrough Fraction")
    axes[1].set_xlim((voltage_range[0], voltage_range[-1]))
    axes[1].set_ylim((0, 1))
    axes[1].plot(np.array(voltage_range[:i+1]), np.array(passthrough[:i+1]), color="m")
    
    axes[2].set_xlabel("Voltage")
    axes[2].set_ylabel("Coherence Distance")
    axes[2].set_xlim((voltage_range[0], voltage_range[-1]))
    axes[2].set_ylim((0, max(max(coherence[:i+1]), 1) * 1.1))
    axes[2].plot(np.array(voltage_range[:i+1]), np.array(coherence[:i+1]), color="g")
        
    video.draw_frame()
    
video.make_movie("beam_passthrough.mp4")